---
title:  "`R` utils"
---

Utils for using R with python in Jupyter

You need to install depending package with `pip install rpy2 rpy2-arrow`

In [ ]:
#| default_exp utils/r

In [ ]:
#| export
import rpy2
import rpy2_arrow.arrow as pyra
from rpy2.robjects.packages import importr
import polars as pl

In [ ]:
#| export
def py2rpy_polars():
    "Helper functions to convert between `polars` and `R` dataframes"
    base = importr('base')

    conv_pl = rpy2.robjects.conversion.Converter(
        'Pandas to pyarrow',
        template=pyra.converter)

    @conv_pl.py2rpy.register(pl.DataFrame)
    def py2rpy_pandas(dataf: pl.DataFrame):
        pa_tbl = dataf.to_arrow()
        return base.as_data_frame(pa_tbl)
        # return pyra.converter.py2rpy(pa_tbl) # NOTE: not working for ggplot2

    conv_pl = rpy2.ipython.rmagic.converter + conv_pl
    return conv_pl

Here is how you can use it, remember to load the extension first:

```{python}
%load_ext rpy2.ipython
from ids_finder.utils.r import py2rpy_polars
conv_pl = py2rpy_polars()
```

Then you can use it like this to export a dataframe `df` to R:

```{python}
%R -i df -c conv_pl
```

In [ ]:
#| eval: false
#| hide
df = pl.DataFrame({
    'a': [1, 2, 3],
    'b': [4, 5, 6],
    'c': ['a', 'b', 'c']
})

%load_ext rpy2.ipython
conv_pl = py2rpy_polars()

%R print(df)